# Calibration - Maximal Intensity
#### v2.0 | by André Maia Chagas, Katrin Franke, and Thomas Euler

This script allows the user to read in spectral data from an Ocean Optics spectometer via USB (i.e. with the USB2000+ spectrometer, Ocean Optics) and convert the data into photoisomerization rates per cone photoreceptor type and LED. For details, see xxxx.

The spectrometer measurements rely on the open source library [python-seabreeze](https://github.com/ap--/python-seabreeze) written by Andreas Pohlmann. This library is a wrapper for the C++ API provided by Ocean Optics. For installation instructions and further support, see there.

*__Note:__ This script is used for the dichromatic (mouse) version of the visual stimulator; it can be easily adapted to other animals with more spectral sensitivites and light source/filter combinations.*

In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy.optimize import curve_fit
from IPython import display

# Spectrometer support
#
import seabreeze.spectrometers as sb

# Helpers
#
import scripts.spectrum as spectrum
import scripts.progress_bar as pbar
from scripts.spectrometer_helper import load_calibration_curves

# Set graphics to be plotted in the notebook
#
%matplotlib inline 

# Seaborn plot settings
#
sns.set()

# Paths and file names
# (Don't change unless you know what you are doing)
#
path_LightSources   = "light-sources//"
path_Filters        = "filters//"
path_SpectCalibData = "spectrometer-calibration-files//"
path_Data           = "data//"
path_Opsins         = "opsins//"

txtFileNewLineStr   = "\r\n"

def setPlotStyle():
    # Presettings for figures
    #
    mpl.rcParams['figure.figsize'] = [10, 5]
    mpl.rc('font', size=10)          
    mpl.rc('axes', titlesize=12)    
    mpl.rc('axes', labelsize=12)    
    mpl.rc('xtick', labelsize=12)   
    mpl.rc('ytick', labelsize=12)   
    mpl.rc('legend', fontsize=12)   
    mpl.rc('figure', titlesize=12) 

### Define photoreceptors

Load opsin spectra from text files in the respective folder:
* Text files are expected to have one column per opsin ... 
* ... plus one (the last one) for the scaling in x direction (wavelength, in 1-nm increments).
* All spectral files, including the filter and LED files loaded later, must have the same x range. In this version, 300 .. 699 nm is used.

In [ ]:
x_wavelen_nm = np.loadtxt(path_Opsins +"mouse_cone_opsins.txt", usecols=(2,)) # 300 .. 699 nm
mouseMOpsin  = np.loadtxt(path_Opsins +"mouse_cone_opsins.txt", usecols=(1,))
mouseSOpsin  = np.loadtxt(path_Opsins +"mouse_cone_opsins.txt", usecols=(0,))

Define someproperties as well as constants needed later to convert the data measured with the spectrometer into photoisomerizations.

*   `h`, Planck's constant [eV*s]
*   `c`, speed of light [m/s]
*   `eV_per_J`, conversion factor, [eV] per [J]
*   `ac_um2`, cone OS light collection area in [µm^2], see [Nikonov et al., 2006](http://www.ncbi.nlm.nih.gov/pubmed/16567464) for details. This is an experimentally determined value, e.g. for wt mouse cones that is fully dark-adapted, a value of 0.2 can be assumed.
*   `ar_um2`, rod OS light collection area, see above. A value of 0.5 is considered realistic.   
    

In [ ]:
h        = 4.135667E-15 # Planck's constant [eV*s]
c        = 299792458    # speed of light [m/s]
eV_per_J = 6.242E+18    # [eV] per [J]
ac_um2   = 0.2     
ar_um2   = 0.5

Organise photoreceptors as a list of dictionaries, with:

*   `name`, name of photoreceptor, used later for plots etc.
*   `peak_nm`, peak wavelength of opsin spectrum in [nm]          
*   `collecArea_um2`, see above
*   `spect`, opsin spectrum

In [ ]:
MCone = {"name"           : "mouse_M_cone",
         "peak_nm"        : 511,
         "collecArea_um2" : ac_um2, 
         "spect"          : mouseMOpsin}
SCone = {"name"           : "mouse_S_cone",
         "peak_nm"        : 360,         
         "collecArea_um2" : ac_um2, 
         "spect"          : mouseSOpsin}
Rod   = {"name"           : "mouse_rod",
         "peak_nm"        : 510,         
         "collecArea_um2" : ar_um2, 
         "spect"          : []}

PRs   = [MCone, SCone, Rod]

Plot cone photoreceptor spectra:

In [ ]:
setPlotStyle()

for PR in PRs:
    if len(PR["spect"]) > 0:
        col = spectrum.wavelength_to_rgb(PR["peak_nm"], darker=0.75)
        plt.plot(x_wavelen_nm, PR["spect"], color=col, label=PR["name"])

plt.gca().set(xlabel="wavelength [nm]", ylabel="rel. sensitivity") 
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")

### Define stimulus LED/filter combinations 
Load LED and filter (dicroic) spectra from text files in the respective folder. Files are expected to be organised as the photoreceptor data files except that they contain only two columns (wavelength, from 300 to 699 nm in 1-nm increments; relative transmission for filters and relative intensity for light sources).

In [ ]:
# LED spectra
#
LED_Green_Name      = "LED_Green_590"
LED_Green           = np.loadtxt(path_LightSources +LED_Green_Name +".txt", usecols=(0,))
LED_UV_Name         = "LED_UV_385"
LED_UV              = np.loadtxt(path_LightSources +LED_UV_Name +".txt", usecols=(0,))

# Dicroic above the objective lens in the MOM
# (if not used, comment line that loads text file and uncomment next line)
#
Dichr_Obj_name     = "Filter_F73-063_z400-580-890"
Dichr_Obj          = np.loadtxt(path_Filters +Dichr_Obj_name +".txt", usecols=(0,))
#Dichr_Obj         = np.array(list([1.0]*len(LED_UV)))

... if one filter per LED is used, uncomment this part.


In [ ]:
%%capture
# LED filter(s)
#
Filter_UV_Name      = "Filter_387_11"
Filter_UV           = np.loadtxt(path_Filters +Filter_UV_Name +".txt", usecols=(0,))
Filter_Green_Name   = "Filter_576_10"
Filter_Green        = np.loadtxt(path_Filters +Filter_Green_Name +".txt", usecols=(0,))

... if a dual-band filter is used, uncomment this part. Then the same filter is used for both LEDs.

In [ ]:
%%capture
# LED filter(s)
# (here, one two-band-pass filter is used instead of one for each LED. Therefore,)
#
"""
Filter_UVGreen_name = "Filter_F59-003_390_575"
Filter_UV           = np.loadtxt(path_Filters +Filter_UVGreen_name +".txt", usecols=(0,))
Filter_Green        = np.loadtxt(path_Filters +Filter_UVGreen_name +".txt", usecols=(0,))
"""

Organise the LED/filter combinations and spectra as a list of dictionary, with:  
*   `name`, name of LED/filter combinations, used later for plots etc.
*   `peak_nm`, peak wavelength of LED/filter combination in [nm]          
*   `LED_spect`, spectrum of LED (same x range as the opsin spectra, that is from 300 to 699 nm, and with 1-nm resolution).
*   `filter_spect`, spectrum of filter
*   `spect`, resulting spectrum, is measured below
*   `spect_nw`, **???**
*   `spect_raw_bkg`, mean background spectrum (LED off or at minimum)
*   `spect_raw`, spectrum of LED at maximum, all trials
*   `spect_raw_avg`, mean spectrum of LED at maximum

In [ ]:
minWLen_nm              = 300
maxWLen_nm              = 699
n                       = maxWLen_nm -minWLen_nm +1
x_wavelen_nm            = np.array([v+minWLen_nm for v in range(n)])

spectrum_green_nw_norm  = None
spectrum_green_nw       = None
spectrum_green_raw_bkg  = None
spectrum_green_raw      = None
spectrum_green_raw_avg  = None

GrLED = {"name"         : "green",
         "peak_nm"      : 576,
         "bandwidth_nm" : 0,
         "LED_spect"    : LED_Green,
         "filter_spect" : Filter_Green, 
         "spect"        : spectrum_green_nw_norm,
         "spect_nw"     : spectrum_green_nw,
         "spect_raw_bkg": spectrum_green_raw_bkg,
         "spect_raw"    : spectrum_green_raw,
         "spect_raw_avg": spectrum_green_raw_avg}

spectrum_uv_nw_norm     = None
spectrum_uv_nw          = None
spectrum_uv_raw_bkg     = None
spectrum_uv_raw         = None
spectrum_uv_raw_avg     = None

UVLED = {"name"         : "UV",
         "peak_nm"      : 387,
         "bandwidth_nm" : 0,
         "LED_spect"    : LED_UV,
         "filter_spect" : Filter_UV,
         "spect"        : spectrum_uv_nw_norm,
         "spect_raw_bkg": spectrum_uv_raw_bkg,
         "spect_raw"    : spectrum_uv_raw,
         "spect_raw_avg": spectrum_uv_raw_avg}

LEDs  = [GrLED, UVLED]

Plot theoretical spectra of LED/filter combinations:

In [ ]:
for LED in LEDs:  
    colLED_lo = spectrum.wavelength_to_rgb(LED["peak_nm"], darker=0.5)
    colLED_hi = spectrum.wavelength_to_rgb(LED["peak_nm"], darker=0.8)
    plt.plot(x_wavelen_nm, LED["LED_spect"], color=colLED_lo, 
             label="LED_" +LED["name"], linewidth=0.5)
    plt.plot(x_wavelen_nm, LED["filter_spect"], color=colLED_hi, 
             label="Filter_" +LED["name"], linewidth=1)
    plt.fill(x_wavelen_nm, LED["LED_spect"], facecolor=colLED_hi, alpha=0.25)    
    
plt.gca().set(xlabel="wavelength [nm]", ylabel="") 
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")

Plot theoretical spectra of LED/filter combinations together with cone photoreceptors:

In [ ]:
for LED in LEDs:  
    col_lo = spectrum.wavelength_to_rgb(LED["peak_nm"], darker=0.5)
    col_hi = spectrum.wavelength_to_rgb(LED["peak_nm"], darker=0.8)
    plt.plot(x_wavelen_nm, LED["LED_spect"], color=col_lo, 
             label="LED_" +LED["name"], linewidth=0.5)
    plt.fill(x_wavelen_nm, LED["LED_spect"], facecolor=col_hi, alpha=0.25)        
    plt.plot(x_wavelen_nm, LED["filter_spect"], color=col_hi, 
             label="Filter_" +LED["name"], linewidth=1)
    
for PR in PRs:
    if len(PR["spect"]) > 0:
        col = spectrum.wavelength_to_rgb(PR["peak_nm"], darker=0.75)
        plt.plot(x_wavelen_nm, PR["spect"], color=col, label=PR["name"])

plt.gca().set(xlabel="wavelength [nm]", ylabel="") 
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")

### Import calibration file for spectrometer
For each wavelength, the file contains corresponding [µJ/count] correlation, which will be used to calibrate the readings from the spectrometer.  
*__Note:__ These files need to be obtained for each individual device.*

In [ ]:
spm_fileName     = path_SpectCalibData +"USB2+F02461_20171108_CC.Irradcal" 
spm_serialNumber = "USB2+F02461"

(spm_calib_values, 
 spm_calib_wavelengths) = load_calibration_curves(spm_fileName, spm_serialNumber)

Plot spectometer calibration curve

In [ ]:
plt.plot(spm_calib_wavelengths, spm_calib_values, color="black", label=spm_serialNumber)
plt.gca().set(xlabel="Wavelength [nm]", ylabel="[µJ/count]")     
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")

### Connect and setup the spectrometer
Select the device using its serial number and set the integration time in [µs]. To obtain [µJ/s], use 1 s.  
*__Note:__ The integration time should be changed if output is saturated.*


In [ ]:
try:
    # Connect to device
    #
    spectrometer = sb.Spectrometer.from_serial_number(spm_serialNumber)
     
    # Set integration time
    #
    int_time_ms = 1.0 *1E6 
    spectrometer.integration_time_micros(int_time_ms)

    # Configure device not to wait for triggers
    #
    spectrometer.trigger_mode(0)  
    
    print("SUCCESS")
    
except sb.SeaBreezeError as Err:
    print("ERROR:", Err)
except NameError:
    print("Could not connect to spectrometer")
    
# Number of trials for measuring background and LED spectra for averaging
#
trials_background = 10 
trials_spectra    = 10    

### Record spectra of light sources
Prepare the measurements and when ready, press Enter. For each LED (light source), first the background spectrum (i.e. LED off or at minimum intensity) and then the spectrum for maximal (or the chosen) intensity is recorded. The recorded mean spectra together with the wavelengts are written to a text file, one per LED. By chosing the name of an existing set of files (`fileNameMask`) in the folder `data//`, no measurements are taking place and instead the data of that earlier recording is loaded.

In [ ]:
fileNameMask = "TFTMonitor_Samsung_10trials"

for iLED, LED in enumerate(LEDs):
    # Check if file with the mask given above does already exist
    #
    fName         = "{0}{1}_{2}.txt".format(path_Data, fileNameMask, LED["name"])
    doesFileExist = os.path.isfile(fName)
    
    # Define colors for the current LED
    #
    col_lo = spectrum.wavelength_to_rgb(LED["peak_nm"], darker=0.5)
    col_hi = spectrum.wavelength_to_rgb(LED["peak_nm"], darker=0.8)

    # LED background:
    #
    if not doesFileExist:
        # Measure ...
        #
        input("Measure background of {0} LED. Press Enter to start ...".format(LED["name"]))
        curr_background = []    
        for i in pbar.log_progress(range(trials_background), name='Background, {0} LED'.format(LED["name"])):
            wavelengths, intensities = spectrometer.spectrum()
            curr_background.append(intensities)
        LED["spect_raw_bkg"] = np.mean(curr_background, axis=0)
    
    else:
        # ... or read data from file
        #
        if i == 0:
            wavelengths = np.loadtxt(fName, usecols=(0,))    
        LED["spect_raw_bkg"] = np.loadtxt(fName, usecols=(1,))

    # Plot ...
    #
    plt.figure(1)  
    plt.plot(wavelengths, LED["spect_raw_bkg"], color=col_lo, label=LED["name"] +" LED, background")
    plt.gca().set(xlabel="Wavelength [nm]", ylabel="Counts (raw spectrometer output)")     
    plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")   
    display.display(plt.gcf())
    display.clear_output(wait=True)
    
    # LED at maximal intensity:
    #
    if not(os.path.isfile(fName)):
        # Measure ...
        #
        input("Prepare to measure maximal intensity {0} LED. Press Enter to continue...".format(LED["name"]))
        curr_spectrum    = []
        LED["spect_raw"] = []
        for i in pbar.log_progress(range(trials_spectra), name='Maximal intensity, {0} LED'.format(LED["name"])):
            wavelengths, intensities = spectrometer.spectrum()
            curr_spectrum = intensities -LED["spect_raw_bkg"]
            LED["spect_raw"].append(curr_spectrum)
        LED["spect_raw_avg"] = np.mean(LED["spect_raw"], axis=0)    
    
    else:
        # .. or read data from file
        #
        LED["spect_raw_avg"] = np.loadtxt(fName, usecols=(2,))
    
    # Plot ...
    #
    plt.figure(2) 
    plt.plot(wavelengths, LED["spect_raw_avg"], color=col_hi, label=LED["name"] +" LED, maximal intensity")
    plt.fill(wavelengths, LED["spect_raw_avg"], facecolor=col_hi, alpha=0.25)        
    plt.gca().set(xlabel="Wavelength [nm]", ylabel="Counts (raw spectrometer output)")     
    plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")   
    display.display(plt.gcf())
    display.clear_output(wait=True)
    
    # Save data for the current LED
    #
    if len(fileNameMask) > 0:
        np.savetxt(fName, np.transpose([wavelengths, LED["spect_raw_bkg"], LED["spect_raw_avg"]]),
                   header="wavelength\traw_bkg\traw_avg", newline=txtFileNewLineStr)    

#### Gauss fit spectra

In [ ]:
led_width = 10

def Gauss(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

led_peak_uv = 387
x = wavelengths
y = spectrum_uv
a = y.max()
mean = led_peak_uv
sigma = led_width

popt, pcov = curve_fit(Gauss,x,y,p0=[a,mean,sigma])
fit_spectrum_uv = Gauss(x,popt[0],popt[1],popt[2])

led_peak_green = 576
x = wavelengths
y = spectrum_green
a = y.max()
mean = led_peak_green
sigma = led_width

popt, pcov = curve_fit(Gauss,x,y,p0=[a,mean,sigma])
fit_spectrum_green = Gauss(x,popt[0],popt[1],popt[2])

plt.plot(wavelengths[0:800],spectrum_green[0:800],color="g",label="Spectrum green LED")
plt.plot(wavelengths[0:800],fit_spectrum_green[0:800],color="r",label="Gauss fits")
plt.plot(wavelengths[0:800],spectrum_uv[0:800],color="b",label="Spectrum UV LED")
plt.plot(wavelengths[0:800],fit_spectrum_uv[0:800],color="r")
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Counts (raw output of spectrometer)")

In [ ]:
... print LED spectra

In [ ]:
for LED in LEDs:  
    colLED_lo = spectrum.wavelength_to_rgb(LED["peak_nm"], darker=0.5)
    colLED_hi = spectrum.wavelength_to_rgb(LED["peak_nm"], darker=0.8)
    plt.plot(x_wavelen_nm, LED["LED_spect"], color=colLED_lo, label="LED_" +LED["name"], linewidth=0.5)
    plt.plot(x_wavelen_nm, LED["filter_spect"], color=colLED_hi, label="Filter_" +LED["name"], linewidth=1)
    plt.fill(x_wavelen_nm, LED["LED_spect"], facecolor=colLED_hi, alpha=0.25)    
    #plt.plot(x_wavelen_nm, LED["spect"], color=colLED, label="Spectrum_" +LED["name"])
    
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")
plt.xlabel("Wavelength (nm)")
plt.ylabel("")

### WHERE?

Define the device detection area in [µm²].  
*__Note::__ ??????*

In [ ]:
r_stimulus_um = 250
A_detect_um2  = math.pi*(r_stimulus_um)**2

#### Transform counts into Watts

In [ ]:
uv_nw = (np.multiply(fit_spectrum_uv,calib_values)/int_time_s)*1000
green_nw = (np.multiply(fit_spectrum_green,calib_values)/int_time_s)*1000

# resample spectra to 300...700 nm
start_nm = int(round(wavelengths[1]))
spectrum_uv_nw = list()
spectrum_green_nw = list()

for iS in range(700-300):
    if iS < (start_nm-300):
        spectrum_uv_nw.append(0)
        spectrum_green_nw.append(0)
    elif iS >= 39:
        index = np.where(wavelengths>=(300+iS))
        spectrum_uv_nw.append(uv_nw[index[0][0]])
    
        index = np.where(wavelengths>=(300+iS))
        spectrum_green_nw.append(green_nw[index[0][0]])

ymax = np.amax(spectrum_uv_nw)
spectrum_uv_nw_norm = spectrum_uv_nw/ymax
spectrum_uv_nw = np.array(spectrum_uv_nw, dtype=float)
spectrum_uv_nw_norm = np.array(spectrum_uv_nw_norm, dtype=float)

ymax = np.amax(spectrum_green_nw)
spectrum_green_nw_norm = spectrum_green_nw/ymax
spectrum_green_nw = np.array(spectrum_green_nw, dtype=float)
spectrum_green_nw_norm = np.array(spectrum_green_nw_norm, dtype=float)

plt.plot(wavelengths[0:800],uv_nw[0:800],color="b",label="UV LED")
plt.plot(wavelengths[0:800],green_nw[0:800],color="g",label="green LED")
plt.xlabel("Wavelength (nm)")
plt.ylabel("nWatts")
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")

print(np.sum(spectrum_green_nw))
print(np.sum(spectrum_uv_nw))



# Determine effective photoreceptor stimulation

### Calculate spectra for effective LED/filter combinations 

In [ ]:
for LED in LEDs:
    LED["effect_on_PR"] = []
    for PR in PRs:
        if len(PR["spect"]) > 0:
            temp = {}
            temp["PR_name"] = PR["name"]
            temp["spect"]   = PR["spect"] * LED["spect"]
            #A_PR            = np.trapz(PR["spect"])
            A_overlap       = np.trapz(temp["spect"])
            A_LED           = np.trapz(LED["spect"]) 
            temp["rel_exc"] = A_overlap/A_LED
            LED["effect_on_PR"].append(temp)

Plot spectra of photoreceptors and LED/filter combinations

In [ ]:
for PR in PRs:
    if len(PR["spect"]) > 0:
        plt.plot(x_wavelen_nm, PR["spect"], 
                 color=scl.wavelength_to_rgb(PR["peak_nm"]), label=PR["name"])
        
for LED in LEDs:        
    if len(LED["spect"]) > 0:    
        colLED = scl.wavelength_to_rgb(LED["peak_nm"])
        plt.plot(x_wavelen_nm, LED["spect"], color=colLED, label="LED_" +LED["name"])
        for effect in LED["effect_on_PR"]:
            for PR in PRs:
                if PR["name"] == effect["PR_name"]:
                    colPR = scl.wavelength_to_rgb(PR["peak_nm"])
            plt.fill(x_wavelen_nm, effect["spect"], facecolor=colPR, alpha=0.25)    

#sns.axlabel("wavelength [nm]", "rel. sensitivity or norm. transmission")     
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")

# Print co-excitation values for all LED/filter and photoreceptor combinations
#
print("Relative co-excitation:")
for LED in LEDs:        
    if len(LED["spect"]) > 0:    
        for effect in LED["effect_on_PR"]:
            print("{0:6.1f}% of {1} by {2}"
                  .format(effect["rel_exc"]*100, effect["PR_name"], LED["name"]))    

Calculate photo-isomerization rates for all LED/filter and photoreceptor combinations

In [ ]:
for iLED, LED in enumerate(LEDs):
    # Convert energy flux from [nW] (=readout of photometer, in [nW]) into [eV/s]
    #
    LED["pow_eflux"] = np.array((LED["spect_nw"] *1E-9 *eV_per_J),dtype=float)
    # Calculate the wavelength-dependent photon energy `Q` in [eV]
    #
    LED["pow_Q"]     = np.array((c*h/(x_wavelen_nm*1E-9)),dtype=float)
    # Divide energy flux by the photon energy to get the photon flux `phi`[photons/s] 
    # and then photon flux density `E` [photons/s /µm^2]
    #
    LED["pow_phi"]   = np.divide(LED["pow_eflux"],LED["pow_Q"])
    LED["pow_E"]     = LED["pow_phi"]/A_detect_um2
    # Calculate per photoreceptor ...
    #
    for effect in LED["effect_on_PR"]:
        for iPR, PR in enumerate(PRs):
            if PR["name"] == effect["PR_name"]:
                # ... photon flux per photoreceptor `photon_rate` in [photons/s]
                #
                A_collect = PR["collecArea_um2"]
                effect["photon_rate"]   = LED["pow_E"] *A_collect                
                # ... photoisomerizations [P*/photoreceptor /s]
                #
                effect["photoiso_rate"] = effect["photon_rate"] *effect["rel_exc"]
                effect["photoiso_rate_total"] = np.sum(effect["photoiso_rate"])
                plt.plot(effect["photoiso_rate"])
                if (iLED == 0) and (iPR == 0): 
                    print("Range of photoisomerization rates:")
                print("{0:5.1f} .. {1:5.1f} 10^3 photons/s in {2} for {3}"
                     .format(effect["photoiso_rate_total"]/1000.0, 
                             effect["photoiso_rate_total"]/1000.0, PR["name"], LED["name"]))

In [ ]:
txtFileNewLineStr = "\r\n"
np.savetxt(pathData + "SpectrumGreen_nW.txt", green_nw, fmt="%1.3f", delimiter=', ', newline=txtFileNewLineStr)
np.savetxt(pathData + "SpectrumUV_nW.txt", uv_nw, fmt="%1.3f", delimiter=', ', newline=txtFileNewLineStr)
np.savetxt(pathData + "Waveltngths.txt", wavelengths, fmt="%1.3f", delimiter=', ', newline=txtFileNewLineStr)
np.savetxt(pathData + "SpectraGreen.txt", current_spectra_green, fmt="%1.3f", delimiter=', ', newline=txtFileNewLineStr)
np.savetxt(pathData + "SpectraUV.txt", current_spectra, fmt="%1.3f", delimiter=', ', newline=txtFileNewLineStr)
